In [1]:
import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.core.constants import ARC_LAMPS
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod

ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 513 frames in store '/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 513
[INFO]: total number of frames found 513


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
os.makedirs(masters_path, exist_ok=True)

In [4]:
lamps = [lamp.lower() for lamp in ARC_LAMPS]

arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

marc_paths = []
for lamp in lamps:
    rest_lamps = copy(lamps)
    rest_lamps.remove(lamp)
    
    lamp_query = f"{lamp} & not " + " & not ".join(rest_lamps)

    for camera in arc_analogs.groups:
        arcs = arc_analogs.get_group(camera)
        arcs = arcs.query(lamp_query)
        arc = arcs.iloc[0].to_dict()
        print(arcs.to_string())

        # define master paths
        mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
        # define master frame path
        marc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"marc_{lamp}", camera=arc["camera"])
        warc_path = path.full("lvm_master", drpver=drpver, tileid=arc["tileid"], mjd=masters_mjd, kind=f"wmarc_{lamp}", camera=camera)
        marc_paths.append(marc_path)
        os.makedirs(os.path.dirname(marc_path), exist_ok=True)
        
        # preprocess and detrend arcs
        darc_paths = []
        for arc in arcs.to_dict("records"):
            darc_path = path.full("lvm_anc", drpver=drpver, kind="d", imagetype=arc["imagetyp"], **arc)
            darc_paths.append(darc_path)
    
        # create master arc (2D image)
        imageMethod.create_master_frame(in_images=darc_paths, out_image=marc_path, master_mjd=masters_mjd)
        
        # extract combined (master) arc
        mtrace_path = os.path.join(masters_path, f"lvm-mtrace-{camera}.fits")
        imageMethod.extract_spectra(in_image=marc_path, out_rss=warc_path, in_trace=mtrace_path, method="aperture", aperture=3)

[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b1-00003457.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b1-00003459.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b1-00003460.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b1-00003458.fits


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
243    s    1111  60177  60177      arc  sp1     b1    3457     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003457.fits
347    s    1111  60177  60177      arc  sp1     b1    3459    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003459.fits
407    s    1111  60177  60177      arc  sp1     b1    3460    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003460.fits
426    s    1111  60177  60177      arc  sp1     b1    3458     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b1-00003458.fits


[INFO]: combining 4 frames into master frame
[INFO]: writing master frame to 'lvm-marc_neon-b1.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b2-00003457.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b2-00003460.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b2-00003459.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b2-00003458.fits


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime  neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
157    s    1111  60177  60177      arc  sp2     b2    3457     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003457.fits
200    s    1111  60177  60177      arc  sp2     b2    3460    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003460.fits
249    s    1111  60177  60177      arc  sp2     b2    3459    150.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003459.fits
298    s    1111  60177  60177      arc  sp2     b2    3458     10.0  True  False    False  False  False  False   False  excellent     0      1       0        0  sdR-s-b2-00003458.fits


[INFO]: combining 4 frames into master frame
[INFO]: writing master frame to 'lvm-marc_neon-b2.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b3-00003458.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b3-00003460.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b3-00003457.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-b3-00003459.fits


...

[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z1.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-z2-00003465.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-z2-00003466.fits


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
119    s    1111  60177  60177      arc  sp2     z2    3465    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003465.fits
181    s    1111  60177  60177      arc  sp2     z2    3466    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z2-00003466.fits


[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z2.fits'
[INFO]: input frames: /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-z3-00003465.fits,/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/ancillary/lvm-darc-z3-00003466.fits


    hemi  tileid    mjd   rmjd imagetyp spec camera  expnum  exptime   neon   hgne  krypton  xenon  argon   ldls  quartz    quality  qual  stage  status  drpqual                    name
329    s    1111  60177  60177      arc  sp3     z3    3465    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003465.fits
474    s    1111  60177  60177      arc  sp3     z3    3466    150.0  False  False    False   True  False  False   False  excellent     0      1       0        0  sdR-s-z3-00003466.fits


[INFO]: combining 2 frames into master frame
[INFO]: writing master frame to 'lvm-marc_xenon-z3.fits'
